# Kubernetes Pods

### 1. Describe simple pod by yaml File

File: simple_pod.yaml

**Create single Pod from yaml file**

In [1]:
%%bash
kubectl create -f simple_pod.yaml

pod/nodeapp-pod created


In [2]:
%%bash
kubectl get pods

NAME          READY   STATUS    RESTARTS   AGE
nodeapp-pod   1/1     Running   0          15s


**Get complete Pod definition as yaml**

In [3]:
%%bash
kubectl get pod nodeapp-pod -o yaml

apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: "2020-07-01T13:40:42Z"
  name: nodeapp-pod
  namespace: default
  resourceVersion: "109896"
  selfLink: /api/v1/namespaces/default/pods/nodeapp-pod
  uid: f1891189-0712-4847-8d2a-84b779377d0b
spec:
  containers:
  - image: ardconsulting/test:nodeapp
    imagePullPolicy: IfNotPresent
    name: nodeapp-container
    ports:
    - containerPort: 8091
      protocol: TCP
    resources: {}
    terminationMessagePath: /dev/termination-log
    terminationMessagePolicy: File
    volumeMounts:
    - mountPath: /var/run/secrets/kubernetes.io/serviceaccount
      name: default-token-srt6x
      readOnly: true
  dnsPolicy: ClusterFirst
  enableServiceLinks: true
  nodeName: docker-desktop
  priority: 0
  restartPolicy: Always
  schedulerName: default-scheduler
  securityContext: {}
  serviceAccount: default
  serviceAccountName: default
  terminationGracePeriodSeconds: 30
  tolerations:
  - effect: NoExecute
    key: node.kubernetes.io/not-rea

**Viewing Pods Log**

In [4]:
%%bash
kubectl logs nodeapp-pod


> simple-node@1.0.0 start /usr/app
> node server.js

Server started...


**Instead of Services you can use Port-Forwarding to connect to the pod**

In [6]:
%%bash
kubectl port-forward nodeapp-pod 8080:8091

Process is interrupted.


-> Now try to access in the browser by localhost:8080

### 2. Organizing Pods with Labels

yaml File with Labels: simple_pod2.yaml

In [10]:
%%bash
kubectl create -f simple_pod2.yaml

pod/nodeapp-pod2 created


In [13]:
%%bash
kubectl get pods --show-labels

NAME           READY   STATUS    RESTARTS   AGE   LABELS
nodeapp-pod    1/1     Running   0          14m   <none>
nodeapp-pod2   1/1     Running   0          42s   app=nodetest,env=dev


**Change pod labels manually**

In [14]:
%%bash
kubectl label pod nodeapp-pod env=dev

pod/nodeapp-pod labeled


In [15]:
%%bash
kubectl get pods --show-labels

NAME           READY   STATUS    RESTARTS   AGE     LABELS
nodeapp-pod    1/1     Running   0          19m     env=dev
nodeapp-pod2   1/1     Running   0          5m21s   app=nodetest,env=dev


**View all Pods with specific label**

In [16]:
%%bash
kubectl get pods -l app=nodetest

NAME           READY   STATUS    RESTARTS   AGE
nodeapp-pod2   1/1     Running   0          6m32s


### 3. Namespaces

**Namespaces are like ressource groups**. Ressource Names only need to be unique within a namespace. Namespaces can be used for multi-tenant apps or for seperating dev/prod envs...

In [17]:
%%bash
kubectl get namespaces

NAME                   STATUS   AGE
default                Active   2d
docker                 Active   2d
kube-node-lease        Active   2d
kube-public            Active   2d
kube-system            Active   2d
kubernetes-dashboard   Active   65m


By default **kubectl get** shows ressources from the **default** namespace

See pods in other namespace:

In [18]:
%%bash
kubectl get pods -n kube-system

NAME                                     READY   STATUS    RESTARTS   AGE
coredns-5644d7b6d9-dwcwj                 1/1     Running   0          2d
coredns-5644d7b6d9-v2h4r                 1/1     Running   0          2d
etcd-docker-desktop                      1/1     Running   0          2d
kube-apiserver-docker-desktop            1/1     Running   0          2d
kube-controller-manager-docker-desktop   1/1     Running   0          2d
kube-proxy-2qjfw                         1/1     Running   0          2d
kube-scheduler-docker-desktop            1/1     Running   0          2d
kubernetes-dashboard-7c54d59f66-7zbz8    1/1     Running   0          90m
storage-provisioner                      1/1     Running   0          2d
vpnkit-controller                        1/1     Running   0          2d


**Create custom namespace with yaml file:** custom_ns.yaml

In [19]:
%%bash
kubectl create -f custom_ns.yaml

namespace/custom-namespace created


**Create simple Pod in custom namespace:** simple_pod_custom_ns.yaml

In [20]:
%%bash
kubectl create -f simple_pod_custom_ns.yaml

pod/nodeapp-pod created


In [21]:
%%bash
kubectl get ns

NAME                   STATUS   AGE
custom-namespace       Active   2m6s
default                Active   2d
docker                 Active   2d
kube-node-lease        Active   2d
kube-public            Active   2d
kube-system            Active   2d
kubernetes-dashboard   Active   73m


In [24]:
%%bash
kubectl get pods -n custom-namespace

NAME          READY   STATUS    RESTARTS   AGE
nodeapp-pod   1/1     Running   0          31s


### 4. Deleting Ressources

**Deleting whole namespace**

In [26]:
%%bash
kubectl delete ns custom-namespace

namespace "custom-namespace" deleted


In [27]:
%%bash
kubectl get ns

NAME                   STATUS   AGE
default                Active   2d
docker                 Active   2d
kube-node-lease        Active   2d
kube-public            Active   2d
kube-system            Active   2d
kubernetes-dashboard   Active   75m


**Deleting pod by label**

In [29]:
%%bash
kubectl get pods --show-labels

NAME           READY   STATUS    RESTARTS   AGE   LABELS
nodeapp-pod    1/1     Running   0          36m   env=dev
nodeapp-pod2   1/1     Running   0          22m   app=nodetest,env=dev


In [30]:
%%bash
kubectl delete pod -l app=nodetest

pod "nodeapp-pod2" deleted


In [31]:
%%bash
kubectl get pods --show-labels

NAME          READY   STATUS    RESTARTS   AGE   LABELS
nodeapp-pod   1/1     Running   0          37m   env=dev


**Delete all pods in a namespace**

In [32]:
%%bash
kubectl delete pods --all

pod "nodeapp-pod" deleted


In [33]:
%%bash
kubectl get pods

No resources found in default namespace.


**Deleting all Ressources in a Namespace**

In [34]:
%%bash
kubectl delete all --all

service "kubernetes" deleted
service "nodeapp-http" deleted


### 5. Liveness Probes

Kubernetes can check if a container is still alive through liveness probes. You can specify a liveness probe for each container in the pod’s specification. Kubernetes will periodically execute the probe and restart the container if the probe fails.

**Test Case** "simple-node-error": node app with endpoint / yielding 5 times successful response and error after that

Create image and push to registry...

In [1]:
%%bash
docker login

Authenticating with existing credentials...
Login Succeeded


In [2]:
%%bash
docker build -t ardconsulting/test:nodeapp-error simple-node-error

Sending build context to Docker daemon  22.53kB
Step 1/6 : FROM node:alpine
 ---> 3bf5a7d41d77
Step 2/6 : WORKDIR /usr/app
 ---> Using cache
 ---> 34cb48271eaf
Step 3/6 : COPY ./package.json ./
 ---> Using cache
 ---> b9b742cd06ec
Step 4/6 : RUN npm install
 ---> Using cache
 ---> 997c4cef5093
Step 5/6 : COPY ./ ./
 ---> 162e75788c72
Step 6/6 : CMD npm start
 ---> Running in eefeb7680f06
Removing intermediate container eefeb7680f06
 ---> df3c298aeac0
Successfully built df3c298aeac0
Successfully tagged ardconsulting/test:nodeapp-error


In [4]:
%%bash
docker push ardconsulting/test:nodeapp-error

The push refers to repository [docker.io/ardconsulting/test]
e8a083f44801: Preparing
9bb8f7bb2cf2: Preparing
bbecd61c89a0: Preparing
55aef14c4b0d: Preparing
109988400934: Preparing
0cdeb35eff67: Preparing
312072b77e32: Preparing
3e207b409db3: Preparing
0cdeb35eff67: Waiting
312072b77e32: Waiting
3e207b409db3: Waiting
109988400934: Layer already exists
bbecd61c89a0: Layer already exists
55aef14c4b0d: Layer already exists
9bb8f7bb2cf2: Layer already exists
0cdeb35eff67: Layer already exists
3e207b409db3: Layer already exists
312072b77e32: Layer already exists
e8a083f44801: Pushed
nodeapp-error: digest: sha256:dfb69b1b7f3ebc42f67b3437780a779b80617598b5f15389ecfd0baf22cc49cc size: 1991


**Add Liveness probe** in simpel_pod_liveness.yaml. Set "initialDelaySeconds" to 15 so server has time to start until health checks are performed.

Create pod and use port forwarding...

In [16]:
%%bash
kubectl create -f simple_pod_liveness.yaml

pod/nodeapp-pod created


In [17]:
%%bash
kubectl port-forward node/nodeapp-pod 8080:8091

Process is interrupted.


**Test health check**: go to browser submit request to localhost:8080 until error occurs, than wait and try again...

In [19]:
%%bash
kubectl describe pod nodeapp-pod

Name:         nodeapp-pod
Namespace:    default
Priority:     0
Node:         docker-desktop/192.168.65.3
Start Time:   Fri, 03 Jul 2020 14:54:18 +0200
Labels:       <none>
Annotations:  <none>
Status:       Running
IP:           10.1.0.18
IPs:
  IP:  10.1.0.18
Containers:
  nodeapp-container:
    Container ID:   docker://82028cdb9510ea68d61ae1eec70f68e24c99b73f965534380cbb7bccd5f7795f
    Image:          ardconsulting/test:nodeapp-error
    Image ID:       docker-pullable://ardconsulting/test@sha256:dfb69b1b7f3ebc42f67b3437780a779b80617598b5f15389ecfd0baf22cc49cc
    Port:           8091/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Fri, 03 Jul 2020 14:55:08 +0200
    Last State:     Terminated
      Reason:       Completed
      Exit Code:    0
      Started:      Fri, 03 Jul 2020 14:54:20 +0200
      Finished:     Fri, 03 Jul 2020 14:55:08 +0200
    Ready:          True
    Restart Count:  1
    Liveness:       http-get http://:8091/ delay=15s timeout

In [ ]:
%%bash
kubectl delete all --all

### 6. How to write yaml Files?

Explore specs by **kubectl explain**

In [35]:
%%bash
kubectl explain pod

KIND:     Pod
VERSION:  v1

DESCRIPTION:
     Pod is a collection of containers that can run on a host. This resource is
     created by clients and scheduled onto hosts.

FIELDS:
   apiVersion	<string>
     APIVersion defines the versioned schema of this representation of an
     object. Servers should convert recognized schemas to the latest internal
     value, and may reject unrecognized values. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#resources

   kind	<string>
     Kind is a string value representing the REST resource this object
     represents. Servers may infer this from the endpoint the client submits
     requests to. Cannot be updated. In CamelCase. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#types-kinds

   metadata	<Object>
     Standard object's metadata. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#met

In [36]:
%%bash
kubectl explain pod.metadata

KIND:     Pod
VERSION:  v1

RESOURCE: metadata <Object>

DESCRIPTION:
     Standard object's metadata. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#metadata

     ObjectMeta is metadata that all persisted resources must have, which
     includes all objects users must create.

FIELDS:
   annotations	<map[string]string>
     Annotations is an unstructured key value map stored with a resource that
     may be set by external tools to store and retrieve arbitrary metadata. They
     are not queryable and should be preserved when modifying objects. More
     info: http://kubernetes.io/docs/user-guide/annotations

   clusterName	<string>
     The name of the cluster which the object belongs to. This is used to
     distinguish resources with same name and namespace in different clusters.
     This field is not set anywhere right now and apiserver is going to ignore
     it if set in create or update request.

   creationTimestamp	<strin

In [38]:
%%bash
kubectl explain pod.spec

KIND:     Pod
VERSION:  v1

RESOURCE: spec <Object>

DESCRIPTION:
     Specification of the desired behavior of the pod. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#spec-and-status

     PodSpec is a description of a pod.

FIELDS:
   activeDeadlineSeconds	<integer>
     Optional duration in seconds the pod may be active on the node relative to
     StartTime before the system will actively try to mark it failed and kill
     associated containers. Value must be a positive integer.

   affinity	<Object>
     If specified, the pod's scheduling constraints

   automountServiceAccountToken	<boolean>
     AutomountServiceAccountToken indicates whether a service account token
     should be automatically mounted.

   containers	<[]Object> -required-
     List of containers belonging to the pod. Containers cannot currently be
     added or removed. There must be at least one container in a Pod. Cannot be
     updated.

   dnsConfig	<Obje

In [40]:
%%bash
kubectl explain pod.spec.containers.ports

KIND:     Pod
VERSION:  v1

RESOURCE: ports <[]Object>

DESCRIPTION:
     List of ports to expose from the container. Exposing a port here gives the
     system additional information about the network connections a container
     uses, but is primarily informational. Not specifying a port here DOES NOT
     prevent that port from being exposed. Any port which is listening on the
     default "0.0.0.0" address inside a container will be accessible from the
     network. Cannot be updated.

     ContainerPort represents a network port in a single container.

FIELDS:
   containerPort	<integer> -required-
     Number of port to expose on the pod's IP address. This must be a valid port
     number, 0 < x < 65536.

   hostIP	<string>
     What host IP to bind the external port to.

   hostPort	<integer>
     Number of port to expose on the host. If specified, this must be a valid
     port number, 0 < x < 65536. If HostNetwork is specified, this must match
     ContainerPort. Most container